# Installing Libraries

In [ ]:
!pip install -q scikit-learn==1.4.1.post1 numpy pandas altair

# Ensemble Learning

수업 시간에 다룬 많은 Ensemble 방법론들은 [sklearn.ensemble](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) 패키지 내에 구현되어 있다.

이번 실습에서는 [Iris](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris)나 [make_moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html#sklearn.datasets.make_moons) 대신에, 보다 특성값 공간이 큰 데이터를 활용해보자.

In [ ]:
from sklearn.datasets import load_breast_cancer


data = load_breast_cancer()
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

보다시피, 특성값 공간은 30개로 Iris의 3개, make_moons의 2개보다 훨씬 많다.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedShuffleSplit

X, y = data.data, data.target

I_train, I_test = next(StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42).split(X, y))
X_train, y_train = X[I_train], y[I_train]
X_test, y_test = X[I_test], y[I_test]

[sklearn.model_selection.StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit)은 전체 데이터를 두 데이터 셋으로 나눈다. 한 데이터 셋의 샘플 수는 전체 샘플 수의 **(1 - test_size)**배, 나머지는 전체 샘플 수의 **test_size** 배이다. 이 때, 두 데이터 셋의 클래스 레이블 비율은 동일하게 분배된다. 예를 들어, 전체 데이터의 클래스 레이블의 비율이 0: 0.4  vs. 1: 0.6이라고 하자. 그럼, 이 함수를 이용해서 두 데이터 셋으로 나눠도 각 데이터 셋의 클래스 레이블 비율은 0: 0.4  vs. 1: 0.6으로 같다. 이처럼, 전체 데이터를 나눴을 때, 원래 전체 데이터가 가지고 있는 특성(즉, 클래스 레이블의 비율)은 유지되는 것을 Stratification 이라고 한다.

실제로 비율이 같은지 확인해보자.

In [ ]:
import numpy as np


print('y:', np.unique(y, return_counts=True))
print('y_train:', np.unique(y_train, return_counts=True))
print('y_test:', np.unique(y_test, return_counts=True))

y: (array([0, 1]), array([212, 357]))
y_train: (array([0, 1]), array([127, 214]))
y_test: (array([0, 1]), array([ 85, 143]))


## Bagging
비선형 데이터에 대해서, Decision Tree와 Bagging을 적용한 Decision Tree가 어떻게 차이나는지 확인해보자. Bagging의 scikit-learn 구현체는 [sklearn.ensemble.BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)이다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


model_single = DecisionTreeClassifier(random_state=42).fit(
    X_train, y_train
)

model_bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42), # Weak Learner의 인스턴스
    n_estimators=100, # 훈련시킬 Weak Learner의 개수
    bootstrap=True, # Bootstrap 적용 여부; True 라면 복원 추출이 된다.
    max_samples=0.5, # Weak Learner를 훈련시킬 때 사용할 데이터의 수; 전체 중 50%의 데이터를 추출한다.
    random_state=42
).fit(
    X_train, y_train
)

성능 측정은 개인 과제 #2에서 사용하고 있는 Accuracy를 활용해보자.

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Bagging: ', accuracy_score(y_train, model_bagging.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Bagging: ', accuracy_score(y_test, model_bagging.predict(X_test)))


# Training performance
- Single:  1.0
- Bagging:  0.9912023460410557
# Validation performance
- Single:  0.9298245614035088
- Bagging:  0.9517543859649122


보다시피, 훈련 데이터에 대해서는 Decision Tree가 더 좋은 성능을 나타냈다. 하지만, 검증 데이터에 대해서는 Bagging이 보다 좋은 성능을 내었다.
즉, Decision Tree는 다소 과대적합되었고, Bagging은 일반화 성능이 더 좋을 것이라고 추정할 수 있다.

## Pasting
Pasting을 적용하는 것은 간단하다. [sklearn.ensemble.BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)의 초매개변수 중 하나인 **bootstrap**을 **False**로 바꿔주기만 하면 된다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


model_pasting = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42), # Weak Learner의 인스턴스
    n_estimators=100, # 훈련시킬 Weak Learner의 개수
    bootstrap=False, # Bootstrap 적용 여부; False 라면 비복원 추출이 된다.
    max_samples=0.5, # Weak Learner를 훈련시킬 때 사용할 데이터의 수; 전체 중 50%의 데이터를 추출한다.
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Pasting: ', accuracy_score(y_train, model_pasting.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Pasting: ', accuracy_score(y_test, model_pasting.predict(X_test)))


# Training performance
- Single:  1.0
- Pasting:  0.9970674486803519
# Validation performance
- Single:  0.9298245614035088
- Pasting:  0.9517543859649122


Bagging과 비교하면 Pasting의 경우 아주 조금 더 훈련 데이터 셋에 대해 학습을 잘 했으며, 검증 데이터 셋에 대해서는 같은 성능을 보인다.

## Random Subspace
이번엔 특성값 중 일부만 활용해서 Weak Learner들을 훈련시켜보자.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


model_rs = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42), # Weak Learner의 인스턴스
    n_estimators=100, # 훈련시킬 Weak Learner의 개수
    bootstrap=False, # Bootstrap 적용 여부; False 라면 복원 추출이 된다.
    max_samples=1.0, # Weak Learner를 훈련시킬 때 사용할 데이터의 수; 전체 데이터를 다 활용한다.
    max_features=0.5, # 원본 데이터의 특성값들 중 반만 사용해보자
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Random Subspace: ', accuracy_score(y_train, model_rs.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Random Subspace: ', accuracy_score(y_test, model_rs.predict(X_test)))

# Training performance
- Single:  1.0
- Pasting:  1.0
# Validation performance
- Single:  0.9298245614035088
- Pasting:  0.956140350877193


검증 데이터에 대한 성능이 Pasting과 Bagging 보다 더 올랐음을 확인할 수 있다.

## Random Patch
이번엔 특성값 공간과 샘플들을 모두 무작위로 뽑되, 특성값과 샘플 모두 비복원 추출로 뽑아보자.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


model_rp = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42), # Weak Learner의 인스턴스
    n_estimators=100, # 훈련시킬 Weak Learner의 개수
    bootstrap=True, # 샘플에 대한 Bootstrap 적용 여부; True 라면 비복원 추출이 된다.
    bootstrap_features=True, # 특성값에 대한 Bootstrap 적용 여부; True 라면 비복원 추출이 된다.
    max_samples=0.5, # Weak Learner를 훈련시킬 때 사용할 데이터의 수; 50%의 데이터만 활용한다.
    max_features=0.5, # 원본 데이터의 특성값들 중 반만 사용해보자
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Random Patch: ', accuracy_score(y_train, model_rp.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Random Patch: ', accuracy_score(y_test, model_rp.predict(X_test)))

# Training performance
- Single:  1.0
- Pasting:  0.9912023460410557
# Validation performance
- Single:  0.9298245614035088
- Pasting:  0.956140350877193


훈련 데이터 셋에 대한 성능은 Random Subspace보다 조금 떨어졌고, 검증 데이터 셋에 대한 성능은 유지되었다.

## Voting
이번엔 서로 다른 알고리즘으로 훈련된 Weak Learner들의 예측 결과를 합쳐보자.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


model_hard_voting = VotingClassifier(
    estimators = [
        ('svc-rbf', SVC(kernel='rbf', probability=True, random_state=42)),
        ('svc-poly', SVC(kernel='poly', probability=True, random_state=42)),
        ('dt', DecisionTreeClassifier(random_state=42))
    ], # 일반적인 학습 모델들
    voting='hard' # Hard Voting
).fit(
    X_train, y_train
)

model_soft_voting = VotingClassifier(
    estimators = [
        ('svc-rbf', SVC(kernel='rbf', probability=True, random_state=42)),
        ('svc-poly', SVC(kernel='poly', probability=True, random_state=42)),
        ('dt', DecisionTreeClassifier(random_state=42))
    ], # 일반적인 학습 모델들
    voting='soft' # Soft Voting
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Hard Voting: ', accuracy_score(y_train, model_hard_voting.predict(X_train)))
print('- Soft Voting: ', accuracy_score(y_train, model_soft_voting.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Hard Voting: ', accuracy_score(y_test, model_hard_voting.predict(X_test)))
print('- Soft Voting: ', accuracy_score(y_test, model_soft_voting.predict(X_test)))

# Training performance
- Single:  1.0
- Hard Voting:  0.9149560117302052
- Soft Voting:  0.9589442815249267
# Validation performance
- Single:  0.9298245614035088
- Hard Voting:  0.9078947368421053
- Soft Voting:  0.9342105263157895


Hard Voting은 단일 모델보다 성능이 떨어졌다. 대신 Soft Voting은 성능이 괜찮은 것을 볼 수 있다. 실제로  [sklearn.ensemble.BaggingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html#sklearn.ensemble.BaggingClassifier)의 경우 Weak Learner들이 **predict_proba** 함수를 지원하지 않을때만 Hard Voting이 이용되며, 기본적으로는 Soft Voting을 활용한다.

## Stacking
이번엔 Voting 대신, 서로 다른 알고리즘으로 훈련된 Weak Learner들의 예측 결과를 입력으로 사용하는 또 다른 학습 모델을 사용해보자. 바로 Stacking 기법이다. Stacking은 [sklearn.ensemble.StackingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html#sklearn.ensemble.StackingClassifier) 구현체를 활용하면 쉽게 사용할 수 있다.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression


model_stack = StackingClassifier(
    estimators = [
        ('svc-rbf', SVC(kernel='rbf', probability=True, random_state=42)),
        ('svc-poly', SVC(kernel='poly', probability=True, random_state=42)),
        ('dt', DecisionTreeClassifier(random_state=42))
    ], # 일반적인 학습 모델들
    final_estimator=LogisticRegression(random_state=42) # Meta-learner
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Stacking: ', accuracy_score(y_train, model_stack.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Stacking: ', accuracy_score(y_test, model_stack.predict(X_test)))

# Training performance
- Single:  1.0
- Pasting:  0.9618768328445748
# Validation performance
- Single:  0.9298245614035088
- Pasting:  0.9342105263157895


## Boosting
Boosting은 다음 시간에 Adaptive Boosting과 Gradient Boosting을 배우면서 해보자.

# Random Forest

일반적인 Ensemble Learning 기법 말고, 아예 이에 특화시킨 기계 학습 알고리즘을 활용해보자. 먼저 [sklearn.ensemble.RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)에 구현되어 있는 Random Forest다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier


model_rf = RandomForestClassifier(
    n_estimators=100, # 500개의 Decision Tree를 만든다
    max_depth=5, # 각 Decision Tree의 깊이는 5로 제한한다
    max_features='sqrt', # 30개의 특성값 공간 중 약 5개만 사용한다
    max_samples=0.5, # 전체 훈련 샘플 중 50%만 Bootstrap한다
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Random Forest: ', accuracy_score(y_train, model_rf.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Random Forest: ', accuracy_score(y_test, model_rf.predict(X_test)))

# Training performance
- Single:  1.0
- Random Forest:  0.9882697947214076
# Validation performance
- Single:  0.9298245614035088
- Random Forest:  0.9473684210526315


위에서 사용한 초매개변수 이외에도 [sklearn.tree.DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)에서 활용되는 다양한 초매개변수들을 거의 동일하게 사용할 수 있다. 보다 세부적인 것은 API 문서를 참조하자.

# ExtraTrees
이번엔 Decision Tree의 분할 방법조차도 무작위로 하는 [sklearn.ensemble.ExtraTrees](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier)를 활용해보자.

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier


model_et = RandomForestClassifier(
    n_estimators=100, # 500개의 Decision Tree를 만든다
    max_depth=5, # 각 Decision Tree의 깊이는 5로 제한한다
    max_features='sqrt', # 30개의 특성값 공간 중 약 5개만 사용한다
    max_samples=0.5, # 전체 훈련 샘플 중 50%만 Bootstrap한다
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Single: ', accuracy_score(y_train, model_single.predict(X_train)))
print('- Random Forest: ', accuracy_score(y_train, model_et.predict(X_train)))

print('# Validation performance')
print('- Single: ', accuracy_score(y_test, model_single.predict(X_test)))
print('- Random Forest: ', accuracy_score(y_test, model_et.predict(X_test)))

# Training performance
- Single:  1.0
- Random Forest:  0.9882697947214076
# Validation performance
- Single:  0.9298245614035088
- Random Forest:  0.9473684210526315


보다시피, ExtraTrees의 많은 초매개변수가 Random Forest와 거의 유사하다.

주의: 위에서 보여준 성능의 경향은 어디까지나 여기에 사용한 데이터에 한한 것이다. 절대 일반적인 것이 아니며, 데이터의 성질에 따라 언제든지 달라질 수 있으니 유의해야 한다.